In [1]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [2]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-07-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-07-31", "%Y-%m-%d")

#Create a range of dates to iterate 
#Note: Periods here represents the number of days it will create from the start date 
#We also do a +2 since it will only generate up to April 29. We inlcude May 1 
#since we want to get data from the last day which is April 30 to May 1 
#changed dates to July 1 - 31
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields=["id","author","subreddit","created_utc","num_comments","score","title","selftext"]
subreddit = 'Mavericks'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-07-01 to 2020-07-02
=====Done
Doing 2020-07-02 to 2020-07-03
=====Done
Doing 2020-07-03 to 2020-07-04
=====Done
Doing 2020-07-04 to 2020-07-05
=====Done
Doing 2020-07-05 to 2020-07-06
=====Done
Doing 2020-07-06 to 2020-07-07
=====Done
Doing 2020-07-07 to 2020-07-08
=====Done
Doing 2020-07-08 to 2020-07-09
=====Done
Doing 2020-07-09 to 2020-07-10
=====Done
Doing 2020-07-10 to 2020-07-11
=====Done
Doing 2020-07-11 to 2020-07-12
=====Done
Doing 2020-07-12 to 2020-07-13
=====Done
Doing 2020-07-13 to 2020-07-14
=====Done
Doing 2020-07-14 to 2020-07-15
=====Done
Doing 2020-07-15 to 2020-07-16
=====Done
Doing 2020-07-16 to 2020-07-17
=====Done
Doing 2020-07-17 to 2020-07-18
=====Done
Doing 2020-07-18 to 2020-07-19
=====Done
Doing 2020-07-19 to 2020-07-20
=====Done
Doing 2020-07-20 to 2020-07-21
=====Done
Doing 2020-07-21 to 2020-07-22
=====Done
Doing 2020-07-22 to 2020-07-23
=====Done
Doing 2020-07-23 to 2020-07-24
=====Done
Doing 2020-07-24 to 2020-07-25
=====Done
Doing 2020-07-25

In [3]:
results

[[{'author': 'cashierdude20',
   'created_utc': 1593613585,
   'id': 'hjb7f4',
   'num_comments': 64,
   'score': 3,
   'selftext': '',
   'subreddit': 'Mavericks',
   'title': 'Was anybody else lowkey expecting to see Luka on the cover instead...? I know I was lol'},
  {'author': 'trilltexas',
   'created_utc': 1593632129,
   'id': 'hjhc67',
   'num_comments': 2,
   'score': 1,
   'selftext': '',
   'subreddit': 'Mavericks',
   'title': 'Rick Carlisle first media availability since March: Talks NBA Return, Orlando Bubble, Mavs Practices, National Anthem Protests &amp; Injustice'},
  {'author': 'shaheedmalik',
   'created_utc': 1593566329,
   'id': 'hj0lbu',
   'num_comments': 2,
   'score': 1,
   'selftext': '',
   'subreddit': 'Mavericks',
   'title': '12 of the best Maverick plays so far this season'},
  {'author': 'marvintran76',
   'created_utc': 1593634668,
   'id': 'hji56w',
   'num_comments': 3,
   'score': 1,
   'selftext': '',
   'subreddit': 'Mavericks',
   'title': 'WCS to 

In [4]:
flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())

,author,created_utc,id,num_comments,score,selftext,subreddit,title
0,cashierdude20,1593613585,hjb7f4,64,3,,Mavericks,Was anybody else lowkey expecting to see Luka ...
1,trilltexas,1593632129,hjhc67,2,1,,Mavericks,Rick Carlisle first media availability since M...
2,shaheedmalik,1593566329,hj0lbu,2,1,,Mavericks,12 of the best Maverick plays so far this season
3,marvintran76,1593634668,hji56w,3,1,,Mavericks,"WCS to sit out remainder of season, Trey Burke..."
4,WillyTanner,1593640097,hjjwmh,5,1,I wanted to create a thread dedicated to discu...,Mavericks,Western CONF Players with COVID


In [5]:
#Submissions gathered
#504
df["id"].count()

504

In [6]:
#Author with most submissions
#Roks123 - 33
df["author"].value_counts()

Roks123            33
[deleted]          30
shaheedmalik       20
tpgeek             15
lsmith77            8
                   ..
0bazinga0           1
auroragraphicco     1
ClintJW2000         1
turdnagels          1
PKPCreative         1
Name: author, Length: 280, dtype: int64

In [7]:
#Submission (Title) with highest Score
# JJ on Luka as the leader

df_max_score = df[df.score==df["score"].max()]
# df_max_score
df_max_score["title"]


268    JJ on Luka as the leader ⛹🏼‍♂️
Name: title, dtype: object

In [8]:
#Submission (Title) with most comments
#[GAME THREAD] Dallas Mavericks(40-27) VS Houston Rockets
df_max_comments = df[df.num_comments==df["num_comments"].max()]
# df_max_comments
df_max_comments["title"]

499    [GAME THREAD] Dallas Mavericks(40-27) VS Houst...
Name: title, dtype: object

In [10]:
#Which day of the week has the most submissions

from datetime import datetime
from datetime import date 
import calendar 
from pprint import pprint


def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#extracting utc date of creation into a separate list
utcs = df['created_utc'].to_list()

#converting the utcs list into integers
utcsint = [int(i) for i in utcs] 

#utcsint

#runthrough conversion of UTC into date stamps
converted = []

for i in utcsint:
    converted.append(to_readable_date(i))

#print (converted)

#switch of date formats into strings

dates = (datetime.strptime(ts, '%Y-%m-%d') for ts in converted)
date_strings = [datetime.strftime(d, '%m-%d-%Y') for d in dates]

#pprint(date_strings)

df["Dates"] = date_strings

#df

def findDay(date): 
    dayname = datetime.strptime(date, '%Y-%m-%d').weekday() 
    return (calendar.day_name[dayname]) 

daylist = []

for i in converted:
    daylist.append(findDay(i))
#daylist

df['Posting_Day'] = daylist

#df

Day_mode = df.Posting_Day.mode()


print (Day_mode[0])
##Answer is Friday

# Stats per day
## 4 - highest with 105 submissions

df = pd.to_datetime(df["created_utc"], unit='s')

df.dt.dayofweek

df.dt.dayofweek.value_counts()


Friday


4    105
2     86
3     83
0     71
5     62
6     57
1     40
Name: created_utc, dtype: int64